# Send REST requests to the API running in Aspire

In [ ]:
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [8]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

Name,Value
traceparent,00-170bb7c928f14e3e21a6252cbadd58c2-9f66f98b29819626-00
Name,Value
Date,"Fri, 18 Apr 2025 22:00:59 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,text/plain; charset=utf-8
Content-Length,26


## GET Weather Forecast 🌦️🌈🔆

In [9]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

Name,Value
traceparent,00-bbbea896bc84e18e9640173416717226-5c619e6e1939c90c-00
Name,Value
Date,"Fri, 18 Apr 2025 22:01:04 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,175
requestedRegion,"""Bristol"""
requestedDate,"""2025-04-18T22:01:05.1032594Z"""
requestId,"""da3e7010-6afd-46ec-bacb-dccab2f2a63f"""


## POST Colected Weather Data 🌡️☔💨

In [67]:
// Generate a random reference...
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

elderberry-71

In [26]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Name,Value
traceparent,00-1ea691ae1dabf72b031821e5227c77d7-87a398eddd7f635b-00
Content-Type,application/json
Content-Length,291
Name,Value
Date,"Fri, 18 Apr 2025 22:07:10 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,73
requestId,"""04ddc24d-02b0-4768-ad66-ce5fdfd626ce"""


## GET Traces 🤖

In [ ]:
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object?> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notication Sent")
            thirdPart = $"✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
        
            
        {
            return $"{Resource} | {DisplayName} | {thirdPart}";
        }
        return $"{Resource} | {DisplayName} | {Tags["domain-event.eventclassName"]?? string.Empty}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("https://localhost:7059") };
var response = await client.GetAsync("traces");
var responseJson = await response.Content.ReadAsStringAsync();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();

Error: System.Net.Http.HttpRequestException: No connection could be made because the target machine actively refused it. (localhost:7059)
 ---> System.Net.Sockets.SocketException (10061): No connection could be made because the target machine actively refused it.
   at System.Net.Sockets.Socket.AwaitableSocketAsyncEventArgs.ThrowException(SocketError error, CancellationToken cancellationToken)
   at System.Net.Sockets.Socket.AwaitableSocketAsyncEventArgs.System.Threading.Tasks.Sources.IValueTaskSource.GetResult(Int16 token)
   at System.Net.Sockets.Socket.<ConnectAsync>g__WaitForConnectWithCancellation|285_0(AwaitableSocketAsyncEventArgs saea, ValueTask connectTask, CancellationToken cancellationToken)
   at System.Net.Http.HttpConnectionPool.ConnectToTcpHostAsync(String host, Int32 port, HttpRequestMessage initialRequest, Boolean async, CancellationToken cancellationToken)
   --- End of inner exception stack trace ---
   at System.Net.Http.HttpConnectionPool.ConnectToTcpHostAsy